In [7]:
import time
import numpy as np
import pandas as pd

import Case
from imp import reload
reload(Case)
from tqdm import tqdm

C:\Users\DAHS\AppData\Local\Temp\ipykernel_20588\1818478163.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [8]:
def number(df, mode):
    
    if mode == 'mimic':
        print('number of subject :', len(df.drop_duplicates(subset=["subject_id"])))
        print('number of stay :', len(df.drop_duplicates(subset=["stay_id"])))
        
    else:
        print('number of subject :', len(df.drop_duplicates(subset=["uniquepid"])))
        print('number of stay :', len(df.drop_duplicates(subset=["patientunitstayid"])))

In [31]:
# mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/MIMIC-IV.csv.gz'
# eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eICU.csv.gz'

# mimic = pd.read_csv(mimic_path, compression = 'gzip')
# eicu = pd.read_csv(eicu_path, compression = 'gzip')

In [228]:
mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Make Derived Variable/mimic_df_cp.csv.gz'
mimic = pd.read_csv(mimic_path, compression = 'gzip')

In [235]:
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Make Derived Variable/eicu_df_cp.csv.gz'
eicu = pd.read_csv(eicu_path, compression = 'gzip')

In [236]:
number(mimic, 'mimic')
number(eicu, 'eicu')

number of subject : 15053
number of stay : 16005
number of subject : 13405
number of stay : 14236


In [237]:
mimic_annoted = Case.annotation(mimic, 'mimic')
mimic_annoted = mimic_annoted.reset_index(drop=True)

analysis_cohort_for_mimic = []

idx = mimic_annoted[(mimic_annoted['ABPd']==75.00)|(mimic_annoted['ABPs']==125.00)].index

mimic_annoted['MAP_fillna'].loc[idx]=1
mimic_annoted['ABPd_fillna'].loc[idx]=1
mimic_annoted['ABPs_fillna'].loc[idx]=1

lactic_idx = mimic_annoted[mimic_annoted['Lactate']==1.1].index
mimic_annoted['Lactate_fillna'].loc[lactic_idx]=1
mimic_labeled = Case.optimized_shock_labeler(mimic_annoted, 'mimic')


eicu_annoted = Case.annotation(eicu, 'eicu')
eicu_annoted = eicu_annoted.reset_index(drop=True)

analysis_cohort_for_eicu = []

idx = eicu_annoted[(eicu_annoted['ABPd']==75.00)|(eicu_annoted['ABPs']==125.00)].index

eicu_annoted['MAP_fillna'].loc[idx]=1
eicu_annoted['ABPd_fillna'].loc[idx]=1
eicu_annoted['ABPs_fillna'].loc[idx]=1

lactic_idx = eicu_annoted[eicu_annoted['Lactate']==1.1].index
eicu_annoted['Lactate_fillna'].loc[lactic_idx]=1
eicu_labeled = Case.optimized_shock_labeler(eicu_annoted, 'eicu')

100%|██████████| 14236/14236 [2:15:39<00:00,  1.75it/s]  


In [238]:
mimic_labeled.to_csv('check_point_mimic_labeled_adding_constraint(0312).csv.gz', compression = 'gzip', index=False)
eicu_labeled.to_csv('check_point_eicu_labeled_adding_constraint(0312).csv.gz', compression = 'gzip', index=False)

In [9]:
import pandas as pd
import numpy as np

# mimic_labeled = pd.read_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip')
# eicu_labeled = pd.read_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip')

mimic_labeled = pd.read_csv('check_point_mimic_labeled_adding_constraint(0312).csv.gz', compression = 'gzip')
eicu_labeled = pd.read_csv('check_point_eicu_labeled_adding_constraint(0312).csv.gz', compression = 'gzip')

In [10]:
mimic_labeled['PaO2/FiO2'] = mimic_labeled['PaO2'] / ((mimic_labeled['FIO2 (%)']+ 0.00000001)/100)
eicu_labeled['PaO2/FiO2'] = eicu_labeled['PaO2'] / ((eicu_labeled['FIO2 (%)'] + 0.00000001)/100)

In [11]:
number(mimic_labeled, 'mimic')
number(eicu_labeled, 'eicu')

number of subject : 15053
number of stay : 16005
number of subject : 13405
number of stay : 14236


In [12]:
import Case
from imp import reload
reload(Case)
from tqdm import tqdm

In [13]:
invalid_columns_mimic, _ = Case.find_invalid_columns(mimic_labeled)
invalid_columns_eicu, _ = Case.find_invalid_columns(eicu_labeled)

m_labeled = Case.replace_inf_with_previous(mimic_labeled, invalid_columns_mimic)
e_labeled = Case.replace_inf_with_previous(eicu_labeled, invalid_columns_eicu)

In [14]:
number(m_labeled, 'mimic')
number(e_labeled, 'eicu')

number of subject : 15053
number of stay : 16005
number of subject : 13405
number of stay : 14236


In [20]:
case1_stay_ids = []
for stay_id, group in m_labeled.groupby('stay_id'):
    if all(group['Annotation'] == 'circ') | all(group['Annotation'] == 'ambiguous'):
        case1_stay_ids.append(stay_id)
test_m = m_labeled[~(m_labeled['stay_id'].isin(case1_stay_ids))].reset_index(drop=True)

number(test_m, 'mimic')

number of subject : 14818
number of stay : 15759


In [23]:
case1_stay_ids = []
for stay_id, group in e_labeled.groupby('patientunitstayid'):
    if all(group['Annotation'] == 'circ') | all(group['Annotation'] == 'ambiguous'):
        case1_stay_ids.append(stay_id)
test_e = e_labeled[~(e_labeled['patientunitstayid'].isin(case1_stay_ids))].reset_index(drop=True)

number(test_e, 'eicu')

number of subject : 13169
number of stay : 13975


In [15]:
reload(Case)
m_case1_case2, m_case3_case4 = Case.Case_definetion(m_labeled, 'mimic')
e_case1_case2, e_case3_case4 = Case.Case_definetion(e_labeled, 'eicu')

|Start MIMIC-IV process|
number of subject : 6853
number of stay : 7097
Extract Case 1, Case 2


100%|██████████| 7094/7094 [00:04<00:00, 1681.30it/s]


--------------
Extract Case 3, Case 4


100%|██████████| 7093/7093 [00:46<00:00, 153.39it/s]


|Start eICU-CRD process|
number of subject : 4090
number of stay : 4177
Extract Case 1, Case 2


100%|██████████| 4140/4140 [00:02<00:00, 1402.29it/s]


--------------
Extract Case 3, Case 4


100%|██████████| 4172/4172 [00:10<00:00, 391.58it/s]


In [16]:
number(pd.concat([m_case1_case2, m_case3_case4], axis = 0), 'mimic')
number(pd.concat([e_case1_case2, e_case3_case4], axis = 0), 'eicu')

number of subject : 6853
number of stay : 7097
number of subject : 4090
number of stay : 4177


In [17]:
m_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
m_case3_case4['INDEX'] = 'CASE3_CASE4_DF'

e_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
e_case3_case4['INDEX'] = 'CASE3_CASE4_DF'

In [249]:
mode = 'mimic'
if mode == 'mimic':
        print('|Start MIMIC-IV process|')
        stay_id_id = 'stay_id'
elif mode == 'eicu':
    print('|Start eICU-CRD process|')
    stay_id_id = 'patientunitstayid'

# event 발생 전, ABPd, ABPs를 1번이라도 측정하지 않은 경우 + event set 만 포함
mimic_event = m_case1_case2[m_case1_case2['Case']=='event']
drop_stay = []
print('event가 있는 코호트 수:')
number(mimic_event, 'mimic') 
for stay in mimic_event[stay_id_id].unique():
    view = m_case1_case2[m_case1_case2[stay_id_id]==stay]
    view = view[~(view['Case']=='event')]
    if all(view['ABPd_fillna']==1) | all(view['ABPs_fillna']==1) | all(view['Lactate_fillna']==1):
        drop_stay.append(stay)

filtered = m_case1_case2[~(m_case1_case2['stay_id'].isin(drop_stay))]
target_m_case1_case2 = filtered[filtered.stay_id.isin(mimic_event[stay_id_id].unique())]
print(len(target_m_case1_case2))
number(target_m_case1_case2, 'mimic') 

|Start MIMIC-IV process|
event가 있는 코호트 수:
number of subject : 6850
number of stay : 7094
135163
number of subject : 5111
number of stay : 5213


In [32]:
13975 - - 1225

15200

In [26]:
4056 - 2873

1183

In [18]:
mode = 'eicu'
if mode == 'mimic':
        print('|Start MIMIC-IV process|')
        stay_id_id = 'stay_id'
elif mode == 'eicu':
    print('|Start eICU-CRD process|')
    stay_id_id = 'patientunitstayid'

# event 발생 전, ABPd, ABPs를 1번이라도 측정하지 않은 경우 + event set 만 포함
eicu_event = e_case1_case2[e_case1_case2['Case']=='event']
drop_stay = []
print('event가 있는 코호트 수:', len(eicu_event[stay_id_id].unique()))
number(eicu_event, 'eicu') 
for stay in eicu_event[stay_id_id].unique():
    view = e_case1_case2[e_case1_case2[stay_id_id]==stay]
    view = view[~(view['Case']=='event')]
    if all(view['ABPd_fillna']==1) | all(view['ABPs_fillna']==1) | all(view['Lactate_fillna']==1):
        drop_stay.append(stay)

filtered = e_case1_case2[~(e_case1_case2[stay_id_id].isin(drop_stay))]
target_e_case1_case2 = filtered[filtered[stay_id_id].isin(eicu_event[stay_id_id].unique())]
print(len(target_e_case1_case2))
number(target_e_case1_case2, 'eicu')

|Start eICU-CRD process|
event가 있는 코호트 수: 4140
number of subject : 4056
number of stay : 4140
153165
number of subject : 2873
number of stay : 2915


In [251]:
mimic_analysis = pd.concat([target_m_case1_case2, m_case3_case4[m_case3_case4.stay_id.isin(target_m_case1_case2.stay_id.unique())]], axis = 0).reset_index(drop=True)
eicu_analysis = pd.concat([target_e_case1_case2, e_case3_case4[e_case3_case4.patientunitstayid.isin(target_e_case1_case2.patientunitstayid.unique())]], axis = 0).reset_index(drop=True)

number(mimic_analysis, 'mimic')

number(eicu_analysis, 'eicu')

mimic_ethnicity = {'AMERICAN INDIAN/ALASKA NATIVE': 0, 'ASIAN': 1,
 'ASIAN - ASIAN INDIAN': 2, 'ASIAN - CHINESE': 3,
 'ASIAN - KOREAN': 4, 'ASIAN - SOUTH EAST ASIAN': 5,
 'BLACK/AFRICAN': 6, 'BLACK/AFRICAN AMERICAN': 7,
 'BLACK/CAPE VERDEAN': 8, 'BLACK/CARIBBEAN ISLAND': 9,
 'HISPANIC OR LATINO': 10, 'HISPANIC/LATINO - CENTRAL AMERICAN': 11,
 'HISPANIC/LATINO - COLUMBIAN': 12, 'HISPANIC/LATINO - CUBAN': 13,
 'HISPANIC/LATINO - DOMINICAN': 14, 'HISPANIC/LATINO - GUATEMALAN': 15,
 'HISPANIC/LATINO - HONDURAN': 16, 'HISPANIC/LATINO - MEXICAN': 17,
 'HISPANIC/LATINO - PUERTO RICAN': 18, 'HISPANIC/LATINO - SALVADORAN': 19, 
 'MULTIPLE RACE/ETHNICITY': 20, 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 21, 
 'OTHER': 22, 'PATIENT DECLINED TO ANSWER': 23, 'PORTUGUESE': 24, 'SOUTH AMERICAN': 25,
 'UNABLE TO OBTAIN': 26, 'UNKNOWN': 27, 'WHITE': 28, 'WHITE - BRAZILIAN': 29,
 'WHITE - EASTERN EUROPEAN': 30, 'WHITE - OTHER EUROPEAN': 31, 'WHITE - RUSSIAN': 32}

eicu_ethnicity = {'0': 0, 'African American': 1, 'Asian': 2, 
                  'Caucasian': 3, 'Hispanic': 4, 'Native American': 5, 'Other/Unknown': 6}

new_mimic_ethnicity = {v:k for k,v in mimic_ethnicity.items()}
new_eicu_ethnicity = {v:k for k,v in eicu_ethnicity.items()}

mimic_analysis['ethnicity'] = mimic_analysis['ethnicity'].map(new_mimic_ethnicity)
eicu_analysis['ethnicity'] = eicu_analysis['ethnicity'].map(new_eicu_ethnicity)

A = {'AMERICAN INDIAN/ALASKA NATIVE': 1, 'ASIAN': 2, 'ASIAN - ASIAN INDIAN': 2, 
 'ASIAN - CHINESE': 2, 'ASIAN - KOREAN': 2, 'ASIAN - SOUTH EAST ASIAN': 2, 
 'BLACK/AFRICAN': 5, 'BLACK/AFRICAN AMERICAN': 5, 'BLACK/CAPE VERDEAN': 6, 
 'BLACK/CARIBBEAN ISLAND': 6, 'HISPANIC OR LATINO': 4, 'HISPANIC/LATINO - CENTRAL AMERICAN': 4, 
 'HISPANIC/LATINO - COLUMBIAN': 4, 'HISPANIC/LATINO - CUBAN': 4, 'HISPANIC/LATINO - DOMINICAN': 4, 
 'HISPANIC/LATINO - GUATEMALAN': 4, 'HISPANIC/LATINO - HONDURAN': 4, 'HISPANIC/LATINO - MEXICAN': 4, 
 'HISPANIC/LATINO - PUERTO RICAN': 4, 'HISPANIC/LATINO - SALVADORAN': 4, 'MULTIPLE RACE/ETHNICITY': 6, 
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 6, 'OTHER': 6, 'PATIENT DECLINED TO ANSWER': 6, 'PORTUGUESE': 4, 
 'SOUTH AMERICAN': 5, 'UNABLE TO OBTAIN': 6, 'UNKNOWN': 6, 'WHITE': 3, 'WHITE - BRAZILIAN': 3, 
 'WHITE - EASTERN EUROPEAN': 3, 'WHITE - OTHER EUROPEAN': 3, 'WHITE - RUSSIAN': 3}

B = {'0': 6, 'African American': 1, 'Asian': 2, 'Caucasian': 3, 'Hispanic': 4, 'Native American': 5, 'Other/Unknown': 6}


mimic_analysis['ethnicity'] = mimic_analysis['ethnicity'].map(A)
eicu_analysis['ethnicity'] = eicu_analysis['ethnicity'].map(B)

number of subject : 5111
number of stay : 5213
number of subject : 2873
number of stay : 2915


In [252]:
mimic_analysis['over_lactic'] = (mimic_analysis['Lactate'] >= 2).astype(int)
eicu_analysis['over_lactic'] = (eicu_analysis['Lactate'] >= 2).astype(int)

In [254]:
mimic_result  = pd.DataFrame()

for stay in mimic_analysis['stay_id'].unique():
    view = mimic_analysis[mimic_analysis['stay_id']==stay]

    view.set_index('Time_since_ICU_admission', inplace=True)
    view['MAP_3hr_avg'] = view['MAP'].rolling(window=4, min_periods=1).mean().shift(1) 

    view['MAP_increase'] = (view['MAP'] > view['MAP_3hr_avg']).astype(int)
    view['MAP_decrease'] = (view['MAP'] < view['MAP_3hr_avg']).astype(int)
    view = view.reset_index()

    view = view.fillna(0)
    
    mimic_result = pd.concat([mimic_result, view], axis = 0)

In [268]:
eicu_result  = pd.DataFrame()

for stay in eicu_analysis['patientunitstayid'].unique():
    view = eicu_analysis[eicu_analysis['patientunitstayid']==stay]

    view.set_index('Time_since_ICU_admission', inplace=True)
    view['MAP_3hr_avg'] = view['MAP'].rolling(window=4, min_periods=1).mean().shift(1) 

    view['MAP_increase'] = (view['MAP'] > view['MAP_3hr_avg']).astype(int)
    view['MAP_decrease'] = (view['MAP'] < view['MAP_3hr_avg']).astype(int)
    view = view.reset_index()

    view = view.fillna(0)
    
    eicu_result = pd.concat([eicu_result, view], axis = 0)

In [354]:
def Lactate_up(df, mode = 'mimic'):
    data = df.copy().reset_index(drop=True)
    data['lactate_up'] = 0
    if mode == 'mimic':
        stay_id_id = 'stay_id'
    else:
        stay_id_id = 'patientunitstayid'
    
    for stay in tqdm(data[stay_id_id].unique()):
        interest = data[data[stay_id_id]==stay].sort_values(by='Time_since_ICU_admission')
        idx = interest.index
        interest.set_index('Time_since_ICU_admission', inplace=True)
        interest['Lactate_3hr_avg'] = interest['Lactate'].rolling(window=3, min_periods=1).mean().shift(1) 

        interest['lactate_up'] = (interest['Lactate'] > interest['Lactate_3hr_avg']).astype(int)
        interest['Lactate_down'] = (interest['Lactate'] < interest['Lactate_3hr_avg']).astype(int)
        interest = interest.reset_index()
        interest = interest.fillna(0)
               
        
        data.loc[idx, 'lactate_up'] = interest['lactate_up'].values
        data.loc[idx, 'lactate_down'] = interest['Lactate_down'].values
    return data

mimic_result_lac = Lactate_up(mimic_result, 'mimic')
eicu_result_lac = Lactate_up(eicu_result, 'eicu')

100%|██████████| 2915/2915 [00:08<00:00, 338.83it/s]


In [363]:
def recov_Annotation(data, mode):
    targ = data.copy()
    # targ['progress'] = 0
    
    if mode == 'mimic':
        stay_id_id = 'stay_id'
    else:
        stay_id_id = 'patientunitstayid'
    
    
    for stay_trajectory in tqdm(targ[stay_id_id].unique()):
        interest = targ[targ[stay_id_id] == stay_trajectory]
        
        for idx, row in interest.iterrows():
            if row['INDEX']!='CASE1_CASE2_DF':
                recover_cond = (row['MAP_3hr_avg'] >= 65.0) & ((row['Lactate'] <= 2) | (row['lactate_down']==1)) 
                
                if recover_cond:
                    targ.at[idx, 'after_shock_annotation'] = 'recov'
                else:
                    targ.at[idx, 'after_shock_annotation'] = 'not_recov'
            else:
                targ.at[idx, 'after_shock_annotation'] = 'before_experience_shock'

    return targ

mimic_result_lac_3_4 = recov_Annotation(mimic_result_lac, 'mimic')
eicu_result_lac_3_4 = recov_Annotation(eicu_result_lac, 'eicu')

100%|██████████| 2915/2915 [00:09<00:00, 308.46it/s]


In [404]:
def Case3_Case4_labeler(parts, mode):
    targ = parts.copy().reset_index(drop=True)
    if mode == 'mimic':
        stay_id_id = 'stay_id'
    else:
        stay_id_id = 'patientunitstayid'
    for stayid in tqdm(targ[stay_id_id].unique()):
        interest = targ[targ[stay_id_id]==stayid]
        interest['endpoint_window'] = interest['Time_since_ICU_admission'] + 1
        
        for idx, row in interest.iterrows():
            if row['INDEX']!='CASE1_CASE2_DF':
                current_time = row['Time_since_ICU_admission']
                endpoint_window = row['endpoint_window']

                future_rows = interest[(interest['Time_since_ICU_admission'] > current_time) & (interest['Time_since_ICU_admission'] <= endpoint_window)]

                if any(future_rows['after_shock_annotation'] == 'not_recov'):
                    targ.loc[idx, 'Case'] = 4
                else:
                    targ.loc[idx, 'Case'] = 3

    return targ.reset_index(drop=True)

m_modified = Case3_Case4_labeler(mimic_result_lac_3_4, 'mimic')
e_modified = Case3_Case4_labeler(eicu_result_lac_3_4, 'eicu')

100%|██████████| 2915/2915 [01:12<00:00, 39.96it/s]


In [405]:
interest = m_modified[m_modified['stay_id']==m_modified.stay_id.unique()[24]][['stay_id','Time_since_ICU_admission','MAP', 'MAP_3hr_avg', 'MAP_increase', 'MAP_decrease','Lactate', 'lactate_up', 'lactate_down', 'over_lactic', 'vasoactive/inotropic', 'Annotation', 'progress','after_shock_annotation','Case']]

In [406]:
interest.head(30)

,stay_id,Time_since_ICU_admission,MAP,MAP_3hr_avg,MAP_increase,MAP_decrease,Lactate,lactate_up,lactate_down,over_lactic,vasoactive/inotropic,Annotation,progress,after_shock_annotation,Case
1068,36405440,0,91.666667,0.000000,0,0,1.1,0,0.0,0,0,no_circ,0,before_experience_shock,2.0
1069,36405440,1,91.666667,91.666667,0,0,1.1,0,0.0,0,0,no_circ,0,before_experience_shock,2.0
1070,36405440,2,69.166667,91.666667,0,1,3.4,1,0.0,1,1,ambiguous,0,before_experience_shock,2.0
1071,36405440,3,71.666667,84.166667,0,1,3.4,1,0.0,1,1,circ,0,before_experience_shock,event
1072,36405440,3,71.666667,81.041667,0,1,3.4,1,0.0,1,1,circ,1,not_recov,4
1073,36405440,4,70.333333,76.041667,0,1,3.4,0,0.0,1,1,circ,1,not_recov,4
1074,36405440,5,74.666667,70.708333,1,0,4.7,1,0.0,1,1,circ,1,not_recov,4
1075,36405440,6,65.666667,72.083333,0,1,4.7,1,0.0,1,1,circ,1,not_recov,4
1076,36405440,7,82.000000,70.583333,1,0,4.7,1,0.0,1,1,circ,1,not_recov,4
1077,36405440,8,59.000000,73.166667,0,1,4.7,0,0.0,1,1,circ,1,not_recov,4


In [35]:
(60 + 78) /2

69.0

In [409]:
m_modified.shape

(298318, 236)

In [410]:
e_modified.shape

(241337, 235)

In [411]:
number(m_modified, 'mimic')
number(e_modified, 'eicu')

number of subject : 5111
number of stay : 5213
number of subject : 2873
number of stay : 2915


In [412]:
m_modified[m_modified['Case']!='event'].Case.value_counts(normalize=True).sort_index()

1.0    0.338827
2.0    0.104529
3.0    0.283782
4.0    0.272861
Name: Case, dtype: float64

In [413]:
e_modified[e_modified['Case']!='event'].Case.value_counts(normalize=True).sort_index()

1.0    0.541687
2.0    0.088499
3.0    0.067288
4.0    0.302527
Name: Case, dtype: float64

In [271]:
mimic_result.to_csv('mimic_analysis(new_version0312).csv.gz', compression = 'gzip', index=False)
eicu_result.to_csv('eicu_analysis(new_version0312).csv.gz', compression = 'gzip', index=False)

In [263]:
number(mimic_analysis, 'mimic')
number(eicu_analysis, 'eicu')

number of subject : 5111
number of stay : 5213
number of subject : 2873
number of stay : 2915


In [295]:
case1_m = []
for stay_id, group in m_labeled.groupby('stay_id'):
    if all(group['Annotation'] == 'no_circ'):
        case1_m.append(stay_id)
no_event_mimic = m_labeled[m_labeled['stay_id'].isin(case1_m)].reset_index(drop=True)

In [296]:
case1_e = []
for stay_id, group in e_labeled.groupby('patientunitstayid'):
    if all(group['Annotation'] == 'no_circ'):
        case1_e.append(stay_id)
no_event_eicu = e_labeled[e_labeled['patientunitstayid'].isin(case1_e)].reset_index(drop=True)

In [300]:
number(no_event_mimic, 'mimic')
number(no_event_eicu, 'eicu')

number of subject : 819
number of stay : 824
number of subject : 1878
number of stay : 1894


In [19]:
1878+2873

4751

In [302]:
no_event_mimic['INDEX'] = 'CASE1_CASE2_DF'
no_event_mimic['after_shock_annotation'] = 'before_experience_shock'
no_event_eicu['INDEX'] = 'CASE1_CASE2_DF'
no_event_eicu['after_shock_annotation'] = 'before_experience_shock'

In [301]:
mimic_ethnicity = {'AMERICAN INDIAN/ALASKA NATIVE': 0, 'ASIAN': 1,
 'ASIAN - ASIAN INDIAN': 2, 'ASIAN - CHINESE': 3,
 'ASIAN - KOREAN': 4, 'ASIAN - SOUTH EAST ASIAN': 5,
 'BLACK/AFRICAN': 6, 'BLACK/AFRICAN AMERICAN': 7,
 'BLACK/CAPE VERDEAN': 8, 'BLACK/CARIBBEAN ISLAND': 9,
 'HISPANIC OR LATINO': 10, 'HISPANIC/LATINO - CENTRAL AMERICAN': 11,
 'HISPANIC/LATINO - COLUMBIAN': 12, 'HISPANIC/LATINO - CUBAN': 13,
 'HISPANIC/LATINO - DOMINICAN': 14, 'HISPANIC/LATINO - GUATEMALAN': 15,
 'HISPANIC/LATINO - HONDURAN': 16, 'HISPANIC/LATINO - MEXICAN': 17,
 'HISPANIC/LATINO - PUERTO RICAN': 18, 'HISPANIC/LATINO - SALVADORAN': 19, 
 'MULTIPLE RACE/ETHNICITY': 20, 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 21, 
 'OTHER': 22, 'PATIENT DECLINED TO ANSWER': 23, 'PORTUGUESE': 24, 'SOUTH AMERICAN': 25,
 'UNABLE TO OBTAIN': 26, 'UNKNOWN': 27, 'WHITE': 28, 'WHITE - BRAZILIAN': 29,
 'WHITE - EASTERN EUROPEAN': 30, 'WHITE - OTHER EUROPEAN': 31, 'WHITE - RUSSIAN': 32}

eicu_ethnicity = {'0': 0, 'African American': 1, 'Asian': 2, 
                  'Caucasian': 3, 'Hispanic': 4, 'Native American': 5, 'Other/Unknown': 6}

new_mimic_ethnicity = {v:k for k,v in mimic_ethnicity.items()}
new_eicu_ethnicity = {v:k for k,v in eicu_ethnicity.items()}

no_event_mimic['ethnicity'] = no_event_mimic['ethnicity'].map(new_mimic_ethnicity)
no_event_eicu['ethnicity'] = no_event_eicu['ethnicity'].map(new_eicu_ethnicity)

A = {'AMERICAN INDIAN/ALASKA NATIVE': 1, 'ASIAN': 2, 'ASIAN - ASIAN INDIAN': 2, 
 'ASIAN - CHINESE': 2, 'ASIAN - KOREAN': 2, 'ASIAN - SOUTH EAST ASIAN': 2, 
 'BLACK/AFRICAN': 5, 'BLACK/AFRICAN AMERICAN': 5, 'BLACK/CAPE VERDEAN': 6, 
 'BLACK/CARIBBEAN ISLAND': 6, 'HISPANIC OR LATINO': 4, 'HISPANIC/LATINO - CENTRAL AMERICAN': 4, 
 'HISPANIC/LATINO - COLUMBIAN': 4, 'HISPANIC/LATINO - CUBAN': 4, 'HISPANIC/LATINO - DOMINICAN': 4, 
 'HISPANIC/LATINO - GUATEMALAN': 4, 'HISPANIC/LATINO - HONDURAN': 4, 'HISPANIC/LATINO - MEXICAN': 4, 
 'HISPANIC/LATINO - PUERTO RICAN': 4, 'HISPANIC/LATINO - SALVADORAN': 4, 'MULTIPLE RACE/ETHNICITY': 6, 
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 6, 'OTHER': 6, 'PATIENT DECLINED TO ANSWER': 6, 'PORTUGUESE': 4, 
 'SOUTH AMERICAN': 5, 'UNABLE TO OBTAIN': 6, 'UNKNOWN': 6, 'WHITE': 3, 'WHITE - BRAZILIAN': 3, 
 'WHITE - EASTERN EUROPEAN': 3, 'WHITE - OTHER EUROPEAN': 3, 'WHITE - RUSSIAN': 3}

B = {'0': 6, 'African American': 1, 'Asian': 2, 'Caucasian': 3, 'Hispanic': 4, 'Native American': 5, 'Other/Unknown': 6}


no_event_mimic['ethnicity'] = no_event_mimic['ethnicity'].map(A)
no_event_eicu['ethnicity'] = no_event_eicu['ethnicity'].map(B)

In [303]:
no_event_mimic['over_lactic'] = (no_event_mimic['Lactate'] >= 2).astype(int)
no_event_eicu['over_lactic'] = (no_event_eicu['Lactate'] >= 2).astype(int)

In [304]:
mimic_result_no_event  = pd.DataFrame()

for stay in no_event_mimic['stay_id'].unique():
    view = no_event_mimic[no_event_mimic['stay_id']==stay]

    view.set_index('Time_since_ICU_admission', inplace=True)
    view['MAP_3hr_avg'] = view['MAP'].rolling(window=4, min_periods=1).mean().shift(1) 

    view['MAP_increase'] = (view['MAP'] > view['MAP_3hr_avg']).astype(int)
    view['MAP_decrease'] = (view['MAP'] < view['MAP_3hr_avg']).astype(int)
    view = view.reset_index()

    view = view.fillna(0)
    
    mimic_result_no_event = pd.concat([mimic_result_no_event, view], axis = 0)

In [305]:
eicu_result_no_event  = pd.DataFrame()

for stay in no_event_eicu['patientunitstayid'].unique():
    view = no_event_eicu[no_event_eicu['patientunitstayid']==stay]

    view.set_index('Time_since_ICU_admission', inplace=True)
    view['MAP_3hr_avg'] = view['MAP'].rolling(window=4, min_periods=1).mean().shift(1) 

    view['MAP_increase'] = (view['MAP'] > view['MAP_3hr_avg']).astype(int)
    view['MAP_decrease'] = (view['MAP'] < view['MAP_3hr_avg']).astype(int)
    view = view.reset_index()

    view = view.fillna(0)
    
    eicu_result_no_event = pd.concat([eicu_result_no_event, view], axis = 0)

In [414]:
mimic_result_no_event['progress'] = 0
eicu_result_no_event['progress'] = 0

m_no_event = Lactate_up(mimic_result_no_event, 'mimic')
e_no_event = Lactate_up(eicu_result_no_event, 'eicu')

100%|██████████| 1894/1894 [00:05<00:00, 327.54it/s]


In [415]:
m_no_event['Case'] = 1
e_no_event['Case'] = 1

In [416]:
m_no_event.shape

(92698, 236)

In [417]:
e_no_event.shape

(236163, 235)

In [318]:
number(m_no_event, 'mimic')
number(e_no_event, 'eicu')

number of subject : 819
number of stay : 824
number of subject : 1878
number of stay : 1894


In [418]:
mimic_final = pd.concat([m_no_event[m_modified.columns], m_modified], axis = 0)
eicu_final = pd.concat([e_no_event[e_modified.columns], e_modified], axis = 0)

In [423]:
mimic_final[mimic_final['Case']!='event'].Case.value_counts(normalize=True).sort_index()

1.0    0.497689
2.0    0.079414
3.0    0.215597
4.0    0.207300
Name: Case, dtype: float64

In [424]:
eicu_final[eicu_final['Case']!='event'].Case.value_counts(normalize=True).sort_index()

1.0    0.769753
2.0    0.044460
3.0    0.033804
4.0    0.151983
Name: Case, dtype: float64

In [430]:
eicu_final['HR_fillna'].value_counts()

0    477500
Name: HR_fillna, dtype: int64

In [426]:
mimic_final.to_csv('mimic_analysis(new_version0313).csv.gz', compression = 'gzip', index=False)
eicu_final.to_csv('eicu_analysis(new_version0313).csv.gz', compression = 'gzip', index=False)

In [1]:
import pandas as pd
mimic_final = pd.read_csv('mimic_analysis(new_version0313).csv.gz', compression = 'gzip')

C:\Users\DAHS\AppData\Local\Temp\ipykernel_7492\2995140185.py:2: DtypeWarning: Columns (226) have mixed types. Specify dtype option on import or set low_memory=False.
  mimic_final = pd.read_csv('mimic_analysis(new_version0313).csv.gz', compression = 'gzip')


In [2]:
mimic_final

,Time_since_ICU_admission,Unnamed: 0,ABPd,Cardiac Output_fillna,height_fillna,PEEP,Cardiac Output,PEEP_fillna,Sofa_Liver,Sofa_Cardiovascular,...,Case,lactate_up,progress,after_shock_annotation,INDEX,over_lactic,MAP_3hr_avg,MAP_increase,MAP_decrease,lactate_down
0,0,2999,75.0,1,0,5.0,7.964561,0,2,2,...,1,0,0,before_experience_shock,CASE1_CASE2_DF,0,0.000000,0,0,0.0
1,1,3000,75.0,1,0,5.0,7.964561,0,2,2,...,1,0,0,before_experience_shock,CASE1_CASE2_DF,0,91.666667,0,0,0.0
2,2,3001,75.0,1,0,5.0,7.964561,0,2,2,...,1,0,0,before_experience_shock,CASE1_CASE2_DF,0,91.666667,0,0,0.0
3,3,3002,75.0,1,0,5.0,7.964561,0,2,2,...,1,0,0,before_experience_shock,CASE1_CASE2_DF,0,91.666667,0,0,0.0
4,4,3003,75.0,1,0,5.0,7.964561,0,2,2,...,1,0,0,before_experience_shock,CASE1_CASE2_DF,0,91.666667,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391011,24,172842,52.0,0,0,5.0,6.450000,0,0,2,...,4,0,1,not_recov,CASE3_CASE4_DF,1,74.708333,0,1,0.0
391012,25,172843,54.0,0,0,5.0,6.450000,0,0,2,...,4,0,1,not_recov,CASE3_CASE4_DF,1,73.291667,0,1,0.0
391013,26,172844,77.0,0,0,5.0,5.590000,0,0,2,...,4,0,1,not_recov,CASE3_CASE4_DF,1,72.083333,1,0,0.0
391014,27,172845,78.0,0,0,5.0,5.590000,0,0,2,...,3,0,1,not_recov,CASE3_CASE4_DF,1,74.166667,1,0,0.0


In [39]:

eicu_final = pd.read_csv('eicu_analysis(new_version0313).csv.gz', compression = 'gzip')

C:\Users\DAHS\AppData\Local\Temp\ipykernel_20588\1147647001.py:1: DtypeWarning: Columns (225) have mixed types. Specify dtype option on import or set low_memory=False.
  eicu_final = pd.read_csv('eicu_analysis(new_version0313).csv.gz', compression = 'gzip')


In [40]:
mimic_final[mimic_final['Case']!='event'].Annotation.value_counts(normalize=True).sort_index()

ambiguous    0.404932
circ         0.178047
no_circ      0.417021
Name: Annotation, dtype: float64

In [41]:
eicu_final[eicu_final['Case']!='event'].Annotation.value_counts(normalize=True).sort_index()

ambiguous    0.291613
circ         0.061789
no_circ      0.646599
Name: Annotation, dtype: float64